<a href="https://colab.research.google.com/github/IGleiser/Simulating-Complex-Systems-with-LLM-Driven-Agents/blob/main/multiagentRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from typing import List, Dict, Tuple, Optional
import ipywidgets as widgets
from IPython.display import display

class MarketEnvironment:
    def __init__(
        self,
        n_assets: int = 10,  # Number of stocks
        n_retail: int = 100,  # Number of retail investors
        n_active: int = 5,   # Number of active managers
        n_passive: int = 5,  # Number of passive managers
        n_signals: int = 5,  # Number of signals (k=5: 1M momentum, 12M momentum, value, growth, ESG)
        risk_free_rate: float = 0.02,
        dt: float = 1/12,    # Time step (1 month)
        max_steps: int = 60,  # 5 years
        active_fee: float = 0.015,  # Active manager fee (1.5%)
        passive_fee: float = 0.005,  # Passive manager fee (0.5%)
        info_cost: float = 0.003,  # Information acquisition cost
        retail_rationality: float = 5.0  # Beta parameter for G-learning
    ):
        self.n_assets = n_assets
        self.n_retail = n_retail
        self.n_active = n_active
        self.n_passive = n_passive
        self.n_managers = n_active + n_passive
        self.n_signals = n_signals
        self.rf = risk_free_rate
        self.dt = dt
        self.max_steps = max_steps
        self.active_fee = active_fee
        self.passive_fee = passive_fee
        self.info_cost = info_cost
        self.retail_rationality = retail_rationality

        # Initialize market state
        self.prices = np.ones(n_assets)
        self.time = 0

        # Signal parameters for OU processes
        self.phi = np.array([12.0, 6.0, 2.0, 2.0, 1.0])  # Mean reversion rates for different signals
        self.z_bar = np.zeros((n_assets, n_signals))      # Mean levels of signals
        self.sigma_z = np.array([0.2, 0.15, 0.1, 0.1, 0.05])  # Volatility of signals
        self.omega_z = np.eye(n_signals)  # Correlation matrix for signals

        # Initialize true signals (Ornstein-Uhlenbeck processes)
        self.z = np.random.normal(0, 0.1, (n_assets, n_signals))

        # Market impact parameters
        self.nu = 0.1 * np.ones(n_assets)
        self.a_hat = 0.5 * np.ones(n_assets)
        self.tau = 1.0  # Memory decay parameter for market impact
        self.a_tau = np.zeros(n_assets)  # Exponentially weighted moving avg of trades

        # Return covariance matrix
        self.sigma_r = 0.04 * np.eye(n_assets) + 0.01 * np.ones((n_assets, n_assets))

        # Initialize agents
        self.initialize_agents()

        # History tracking
        self.history = {
            'prices': [self.prices.copy()],
            'returns': [],
            'signals': [self.z.copy()],
            'asset_allocations': [],
            'retail_flows': []
        }

    def initialize_agents(self):
        """Initialize all agents in the market"""
        self.agents = {
            'retail': [RetailInvestor(i, self) for i in range(self.n_retail)],
            'active': [ActiveManager(i, self) for i in range(self.n_active)],
            'passive': [PassiveManager(i + self.n_active, self) for i in range(self.n_passive)]
        }

        # Initialize portfolios - each agent starts with some cash
        for agent_type in self.agents:
            for agent in self.agents[agent_type]:
                agent.cash = 1000.0
                agent.portfolio = np.zeros(self.n_assets)
                if agent_type == 'retail':
                    agent.fund_investments = np.zeros(self.n_managers)

    def update_signals(self):
        """Update the signal processes using OU dynamics"""
        for n in range(self.n_assets):
            # Generate correlated Brownian motion increments
            dW = np.random.multivariate_normal(
                np.zeros(self.n_signals),
                self.omega_z * np.sqrt(self.dt)
            )

            # Update each signal component for asset n
            for k in range(self.n_signals):
                # OU process update: dz = phi * (z_bar - z) * dt + sigma * dW
                self.z[n, k] += self.phi[k] * (self.z_bar[n, k] - self.z[n, k]) * self.dt + self.sigma_z[k] * dW[k]

        # Calculate percentiles for value and growth signals to classify stocks
        value_percentiles = np.argsort(np.argsort(self.z[:, 2])) / self.n_assets
        growth_percentiles = np.argsort(np.argsort(self.z[:, 3])) / self.n_assets

        self.value_growth_diff = value_percentiles - growth_percentiles
        self.stock_labels = np.zeros(self.n_assets, dtype=object)  # Changed to object dtype for string labels
        threshold = 0.3

        # Classify stocks as value, growth, or core
        self.stock_labels[self.value_growth_diff > threshold] = 'value'
        self.stock_labels[self.value_growth_diff < -threshold] = 'growth'
        self.stock_labels[(self.value_growth_diff <= threshold) &
                         (self.value_growth_diff >= -threshold)] = 'core'

        # Calculate ESG percentiles (5th signal)
        self.esg_percentiles = np.argsort(np.argsort(self.z[:, 4])) / self.n_assets

    def compute_market_impact(self, total_trades):
        """Compute market impact based on quadratic impact function"""
        # Update exponentially weighted moving average of trades
        self.a_tau = (1 - self.dt / self.tau) * self.a_tau + (self.dt / self.tau) * total_trades

        # Quadratic impact function: f(a) = nu * a * (a_hat - a)
        impact = self.nu * self.a_tau * (self.a_hat - self.a_tau)

        return impact

    def compute_returns(self, total_trades):
        """Compute asset returns based on signals and market impact"""
        # Expected returns based on signals
        W = np.array([0.2, 0.3, 0.25, 0.25, 0.1])  # Signal weights
        expected_returns = np.zeros(self.n_assets)

        for n in range(self.n_assets):
            expected_returns[n] = np.sum(W * self.z[n, :])

        # Add market impact
        impact = self.compute_market_impact(total_trades)
        expected_returns += impact

        # Add noise
        noise = np.random.multivariate_normal(np.zeros(self.n_assets), self.sigma_r)

        # Total returns (including risk-free rate)
        returns = self.rf * self.dt + expected_returns * self.dt + noise

        return returns


    def step(self):
        """Execute one time step in the market simulation"""
        self.time += 1

        if self.time >= self.max_steps:
            return False  # Simulation ended

        # Update signals
        self.update_signals()

        # Get agent trading decisions
        all_trades = np.zeros(self.n_assets)

        # Passive managers act first
        for agent in self.agents['passive']:
            trades = agent.act(self.z[:, :2])  # Only use first 2 signals
            all_trades += trades

        # Active managers act next
        for agent in self.agents['active']:
            # Active managers have access to all signals
            trades = agent.act(self.z)
            all_trades += trades

        # Retail investors act last
        retail_flows_to_managers = np.zeros(self.n_managers)
        for agent in self.agents['retail']:
            trades, fund_flows = agent.act(self.z[:, :2])  # Only use first 2 signals
            all_trades += trades
            retail_flows_to_managers += fund_flows

        # Apply retail flows to managers
        for i, agent in enumerate(self.agents['active'] + self.agents['passive']):
            agent.receive_flows(retail_flows_to_managers[i])

        # Compute returns based on all trades
        returns = self.compute_returns(all_trades)

        # Update prices
        self.prices *= (1 + returns)

        # Update agent portfolios
        for agent_type in self.agents:
            for agent in self.agents[agent_type]:
                agent.update_portfolio(returns)

        # Record history
        self.history['prices'].append(self.prices.copy())
        self.history['returns'].append(returns)
        self.history['signals'].append(self.z.copy())

        # Record asset allocations
        allocations = {}
        for agent_type in self.agents:
            allocations[agent_type] = np.sum([agent.portfolio for agent in self.agents[agent_type]], axis=0)
        self.history['asset_allocations'].append(allocations)

        # Record retail flows
        self.history['retail_flows'].append(retail_flows_to_managers.copy())

        return True  # Simulation continues

    def run_simulation(self):
        """Run the full simulation"""
        for _ in range(self.max_steps):
            if not self.step():
                break
        return self.history


class Agent:
    def __init__(self, id_num, env):
        self.id = id_num
        self.env = env
        self.cash = 0.0
        self.portfolio = np.zeros(env.n_assets)
        self.wealth_history = []
        self.lambda_risk_aversion = 1.0  # Risk aversion parameter

        # Transaction cost matrix (quadratic costs)
        self.lambda_tc = 0.001 * np.eye(env.n_assets)

    def get_wealth(self):
        """Calculate total wealth"""
        return self.cash + np.sum(self.portfolio * self.env.prices)

    def update_portfolio(self, returns):
        """Update portfolio values after returns are realized"""
        self.portfolio *= (1 + returns)
        self.cash *= (1 + self.env.rf * self.env.dt)
        self.wealth_history.append(self.get_wealth())


class RetailInvestor(Agent):
    def __init__(self, id_num, env):
        super().__init__(id_num, env)
        self.fund_investments = np.zeros(env.n_managers)
        self.beta = env.retail_rationality  # Inverse temperature parameter for bounded rationality

        # Prior policy parameters
        self.prior_mean = np.zeros(env.n_assets + env.n_managers)
        self.prior_cov = 0.1 * np.eye(env.n_assets + env.n_managers)

    def act(self, signals):
        """Make investment decisions based on past returns"""

        # For simplicity, use momentum strategy based on recent returns
        if len(self.env.history['returns']) > 0:
            past_returns = np.array(self.env.history['returns'][-min(3, len(self.env.history['returns'])):])
            avg_returns = np.mean(past_returns, axis=0)
        else:
            avg_returns = np.zeros(self.env.n_assets)

        # Simple mean-variance optimization with bounded rationality
        expected_returns = avg_returns
        covariance = self.env.sigma_r

        # Calculate optimal allocations across assets and funds
        current_allocation = np.concatenate([
            self.portfolio * self.env.prices / max(self.get_wealth(), 1e-6),
            self.fund_investments / max(self.get_wealth(), 1e-6)
        ])

        # Extended returns including funds (assuming fund returns are average of their holdings)
        fund_expected_returns = np.zeros(self.env.n_managers)
        # In reality, we'd compute this from the fund managers' holdings, but we'll simplify

        extended_returns = np.concatenate([expected_returns, fund_expected_returns])

        # Construct extended covariance matrix including funds
        extended_cov = np.zeros((self.env.n_assets + self.env.n_managers,
                               self.env.n_assets + self.env.n_managers))
        extended_cov[:self.env.n_assets, :self.env.n_assets] = covariance

        # Bounded rational decision with stochastic policy
        optimal_allocation = self._bounded_rational_allocation(
            extended_returns, extended_cov, current_allocation
        )

        # Calculate trades to execute
        wealth = self.get_wealth()
        target_asset_holdings = optimal_allocation[:self.env.n_assets] * wealth
        trades = target_asset_holdings - self.portfolio * self.env.prices

        # Calculate fund flows
        target_fund_holdings = optimal_allocation[self.env.n_assets:] * wealth
        fund_flows = target_fund_holdings - self.fund_investments

        # Update cash position and fund investments
        self.cash -= np.sum(trades) + np.sum(fund_flows)
        self.fund_investments += fund_flows

        # Ensure we don't go negative in cash (simplified constraint)
        if self.cash < 0:
            scale = (self.get_wealth() + self.cash) / self.get_wealth()
            trades *= scale
            fund_flows *= scale
            self.cash = 0

        return trades, fund_flows



    def _bounded_rational_allocation(self, expected_returns, covariance, current_allocation):
        """Implement G-learning based bounded-rational allocation"""
        # Parameters for G-learning
        beta = self.beta

        # Get dimensions for the full space (assets + funds)
        n_dim = len(expected_returns)

        # Create transaction cost matrix with correct dimensions
        lambda_tc_extended = np.zeros((n_dim, n_dim))
        # Only apply transaction costs to assets (not funds)
        n_assets = self.env.n_assets
        lambda_tc_extended[:n_assets, :n_assets] = self.lambda_tc

        # Q-function parameters (from the paper's G-learning section)
        Q_xx = -self.lambda_risk_aversion * covariance
        Q_ux = 2 * Q_xx
        Q_uu = Q_xx - lambda_tc_extended  # Use extended transaction cost matrix
        Q_x = expected_returns
        Q_u = Q_x

        # Make sure prior covariance has the right dimension
        if self.prior_cov.shape[0] != n_dim:
            # Create new prior covariance with right dimensions
            self.prior_cov = 0.1 * np.eye(n_dim)
            self.prior_mean = np.zeros(n_dim)

        # Sigma_p from prior policy
        try:
            Sigma_p_inv = np.linalg.inv(self.prior_cov)
        except np.linalg.LinAlgError:
            # Add small regularization if matrix is singular
            Sigma_p_inv = np.linalg.inv(self.prior_cov + 1e-6 * np.eye(n_dim))

        # Calculate G-function optimal policy parameters
        try:
            Sigma_p_tilde_inv = Sigma_p_inv - 2 * beta * Q_uu
            Sigma_p_tilde = np.linalg.inv(Sigma_p_tilde_inv)
        except np.linalg.LinAlgError:
            # Fall back to more stable calculation if matrix is singular
            reg_factor = 1e-3 * np.eye(n_dim)
            Sigma_p_tilde_inv = Sigma_p_inv - 2 * beta * Q_uu + reg_factor
            Sigma_p_tilde = np.linalg.inv(Sigma_p_tilde_inv)

        # Optimal mean calculation
        u_tilde = Sigma_p_tilde @ (Sigma_p_inv @ self.prior_mean + beta * Q_u)

        # Ensure dimensions match
        v_tilde = Sigma_p_tilde @ (Sigma_p_inv @ np.eye(n_dim) + beta * Q_ux)

        # Mean of the policy
        optimal_mean = u_tilde + v_tilde @ current_allocation

        # Sample from the stochastic policy
        try:
            # Ensure the covariance is positive definite
            eigvals = np.linalg.eigvalsh(Sigma_p_tilde)
            if np.min(eigvals) < 1e-6:
                Sigma_p_tilde += 1e-6 * np.eye(n_dim)

            optimal_allocation = np.random.multivariate_normal(
                optimal_mean, Sigma_p_tilde
            )
        except np.linalg.LinAlgError:
            # Fall back to mean without noise if sampling fails
            optimal_allocation = optimal_mean

        # Ensure allocations are reasonable (avoid extreme values)
        optimal_allocation = np.clip(optimal_allocation, -0.5, 1.5)

        # Normalize to ensure sum is 1
        if np.sum(optimal_allocation) > 0:
            optimal_allocation = optimal_allocation / np.sum(optimal_allocation)
        else:
            optimal_allocation = np.ones_like(optimal_allocation) / n_dim

        return optimal_allocation


class AssetManager(Agent):
    def __init__(self, id_num, env):
        super().__init__(id_num, env)
        self.management_fee = 0.01  # 1% annual fee (will be overridden)
        self.clients_capital = 0.0  # Capital invested by retail investors

    def receive_flows(self, flow_amount):
        """Receive capital flows from retail investors"""
        self.clients_capital += flow_amount
        self.cash += flow_amount

    def collect_fees(self):
        """Collect management fees"""
        fee_amount = self.clients_capital * self.management_fee * self.env.dt
        self.clients_capital -= fee_amount
        return fee_amount


class ActiveManager(AssetManager):
    def __init__(self, id_num, env):
        super().__init__(id_num, env)
        self.management_fee = env.active_fee  # Use environment parameter for fee
        self.information_cost = env.info_cost  # Cost of acquiring signals

        # Manager type: value, growth, blend, or esg
        self.manager_type = np.random.choice(['value', 'growth', 'blend', 'esg'])

        # Signal weights - heterogeneous beliefs
        self.W = np.random.uniform(0.8, 1.2, size=(env.n_assets, env.n_signals))

        # Idiosyncratic noise parameters
        self.theta = np.ones((env.n_assets, env.n_signals))
        self.theta[:, 2:4] = np.random.uniform(0.7, 0.9, size=(env.n_assets, 2))  # Heterogeneous signal weights
        self.sigma_hat = np.random.uniform(0.05, 0.15, size=(env.n_assets, env.n_signals))

    def get_private_signals(self, true_signals):
        """Generate noisy, private signals from true signals"""
        # Generate idiosyncratic noise
        z_idiosyncratic = np.zeros((self.env.n_assets, self.env.n_signals))
        for n in range(self.env.n_assets):
            for k in range(2, 4):  # Only value and growth signals have noise
                dW = np.random.normal(0, np.sqrt(self.env.dt))
                z_idiosyncratic[n, k] += -self.env.phi[k] * z_idiosyncratic[n, k] * self.env.dt + self.sigma_hat[n, k] * dW

        # Combine true signals with idiosyncratic noise
        private_signals = np.zeros_like(true_signals)
        for n in range(self.env.n_assets):
            for k in range(self.env.n_signals):
                if k in [2, 3]:  # Value and growth signals
                    private_signals[n, k] = self.theta[n, k] * true_signals[n, k] + (1 - self.theta[n, k]) * z_idiosyncratic[n, k]
                else:
                    private_signals[n, k] = true_signals[n, k]  # No noise for other signals

        return private_signals


    def estimate_expected_returns(self, private_signals):
        """Estimate expected returns based on private signals"""
        expected_returns = np.zeros(self.env.n_assets)

        for n in range(self.env.n_assets):
            # Create a copy of weights to avoid persistent modifications
            W_copy = self.W[n, :].copy()

            # Apply manager type specific weights
            if self.manager_type == 'value':
                W_copy[2] *= 1.5  # Increase weight on value signal
                W_copy[3] *= 0.5  # Decrease weight on growth signal
            elif self.manager_type == 'growth':
                W_copy[2] *= 0.5  # Decrease weight on value signal
                W_copy[3] *= 1.5  # Increase weight on growth signal

            # ESG managers only invest in top ESG stocks
            if self.manager_type == 'esg' and self.env.esg_percentiles[n] < 0.8:
                continue

            expected_returns[n] = np.sum(W_copy * private_signals[n, :])

        return expected_returns

    def act(self, true_signals):
        """Make investment decisions based on private signals"""
        # Get private signals
        private_signals = self.get_private_signals(true_signals)

        # Estimate expected returns
        expected_returns = self.estimate_expected_returns(private_signals)

        # Pay cost for information
        self.cash -= self.information_cost * self.get_wealth() * self.env.dt

        # Calculate optimal portfolio using mean-variance optimization
        covariance = self.env.sigma_r

        # Markowitz optimization with transaction costs
        current_allocation = self.portfolio * self.env.prices / max(self.get_wealth(), 1e-6)
        optimal_allocation = self._markowitz_optimization(expected_returns, covariance, current_allocation)

        # Ensure optimal_allocation has the correct shape
        if len(optimal_allocation) != self.env.n_assets:
            # Pad or truncate to match the number of assets
            if len(optimal_allocation) < self.env.n_assets:
                # Pad with zeros
                padded_allocation = np.zeros(self.env.n_assets)
                padded_allocation[:len(optimal_allocation)] = optimal_allocation
                optimal_allocation = padded_allocation
            else:
                # Truncate
                optimal_allocation = optimal_allocation[:self.env.n_assets]

        # Calculate trades to execute
        wealth = self.get_wealth()
        target_holdings = optimal_allocation * wealth
        trades = target_holdings - self.portfolio * self.env.prices

        # Update cash position
        self.cash -= np.sum(trades)

        # Collect management fees
        fee = self.collect_fees()
        self.cash += fee

        return trades

    def _markowitz_optimization(self, expected_returns, covariance, current_allocation):
        """Implement Markowitz optimization with transaction costs"""
        n = len(expected_returns)

        # Ensure current_allocation has the correct length
        if len(current_allocation) != n:
            # Pad or truncate to match expected returns length
            if len(current_allocation) < n:
                # Pad with zeros
                padded_allocation = np.zeros(n)
                padded_allocation[:len(current_allocation)] = current_allocation
                current_allocation = padded_allocation
            else:
                # Truncate
                current_allocation = current_allocation[:n]

        # Objective: maximize r'x - λ x'Σx - κ (x-x_0)'(x-x_0)
        # where κ represents transaction costs

        # Analytical solution (simplified)
        tc_matrix = 0.1 * np.eye(n)  # Transaction cost scaling
        A = 2 * self.lambda_risk_aversion * covariance + 2 * tc_matrix
        b = expected_returns + 2 * tc_matrix @ current_allocation

        try:
            # Add small regularization to avoid singularity
            A_reg = A + 1e-8 * np.eye(n)
            optimal_allocation = np.linalg.solve(A_reg, b)
        except np.linalg.LinAlgError:
            # Fallback if matrix is singular
            optimal_allocation = current_allocation

        # Apply constraints based on manager type
        if self.manager_type == 'value':
            for i in range(min(len(optimal_allocation), len(self.env.stock_labels))):
                if self.env.stock_labels[i] != 'value' and optimal_allocation[i] > current_allocation[i]:
                    optimal_allocation[i] = current_allocation[i]

        elif self.manager_type == 'growth':
            for i in range(min(len(optimal_allocation), len(self.env.stock_labels))):
                if self.env.stock_labels[i] != 'growth' and optimal_allocation[i] > current_allocation[i]:
                    optimal_allocation[i] = current_allocation[i]

        elif self.manager_type == 'esg':
            for i in range(min(len(optimal_allocation), len(self.env.esg_percentiles))):
                if self.env.esg_percentiles[i] < 0.8:
                    optimal_allocation[i] = 0  # ESG managers don't hold low ESG stocks

        # Normalize to ensure sum is 1 (fully invested)
        if np.sum(optimal_allocation) > 0:
            optimal_allocation = optimal_allocation / np.sum(optimal_allocation)
        else:
            optimal_allocation = np.ones(len(optimal_allocation)) / len(optimal_allocation)  # Equal-weighted fallback

        return optimal_allocation


class PassiveManager(AssetManager):
    def __init__(self, id_num, env):
        super().__init__(id_num, env)
        self.management_fee = env.passive_fee  # Use environment parameter for fee


    def act(self, signals):
        """Make investment decisions based only on public signals"""
        # Passive managers only use public signals (first two components)
        public_signals = signals[:, :2]

        # Simple expected returns model based on public signals
        expected_returns = np.zeros(self.env.n_assets)
        W = np.array([0.6, 0.4])  # Weights for momentum signals

        for n in range(self.env.n_assets):
            expected_returns[n] = np.sum(W * public_signals[n, :])

        # Calculate optimal portfolio using mean-variance optimization
        covariance = self.env.sigma_r

        # Current allocation
        current_allocation = self.portfolio * self.env.prices / max(self.get_wealth(), 1e-6)

        # Passive managers typically have lower trading intensity
        # and try to track some benchmark or follow simple rules
        optimal_allocation = self._simple_optimization(expected_returns, covariance, current_allocation)

        # Ensure optimal_allocation has the correct shape
        if len(optimal_allocation) != self.env.n_assets:
            # Pad or truncate to match the number of assets
            if len(optimal_allocation) < self.env.n_assets:
                # Pad with zeros
                padded_allocation = np.zeros(self.env.n_assets)
                padded_allocation[:len(optimal_allocation)] = optimal_allocation
                optimal_allocation = padded_allocation
            else:
                # Truncate
                optimal_allocation = optimal_allocation[:self.env.n_assets]

        # Calculate trades to execute
        wealth = self.get_wealth()
        target_holdings = optimal_allocation * wealth
        trades = target_holdings - self.portfolio * self.env.prices

        # Update cash position
        self.cash -= np.sum(trades)

        # Collect management fees
        fee = self.collect_fees()
        self.cash += fee

        return trades

    def _simple_optimization(self, expected_returns, covariance, current_allocation):
        """Simplified optimization approach for passive managers"""
        n = len(expected_returns)

        # Ensure current_allocation has the correct length
        if len(current_allocation) != n:
            # Pad or truncate to match expected returns length
            if len(current_allocation) < n:
                # Pad with zeros
                padded_allocation = np.zeros(n)
                padded_allocation[:len(current_allocation)] = current_allocation
                current_allocation = padded_allocation
            else:
                # Truncate
                current_allocation = current_allocation[:n]

        # Passive managers trade less frequently and with lower magnitude
        # We'll use a more conservative approach with higher transaction costs
        tc_matrix = 0.3 * np.eye(n)  # Higher transaction costs than active managers
        A = 2 * self.lambda_risk_aversion * covariance + 2 * tc_matrix
        b = expected_returns + 2 * tc_matrix @ current_allocation

        try:
            # Add regularization to avoid singularity
            A_reg = A + 1e-8 * np.eye(n)
            optimal_allocation = np.linalg.solve(A_reg, b)
        except np.linalg.LinAlgError:
            optimal_allocation = current_allocation

        # Passive managers might implement simpler strategies
        # Here we implement a momentum-based strategy with some mean-reversion
        if len(self.env.history['returns']) > 0:
            recent_returns = self.env.history['returns'][-1]
            # Add a small mean-reversion component
            # Ensure recent_returns has the right length
            if len(recent_returns) != n:
                if len(recent_returns) < n:
                    # Pad with zeros
                    padded_returns = np.zeros(n)
                    padded_returns[:len(recent_returns)] = recent_returns
                    recent_returns = padded_returns
                else:
                    # Truncate
                    recent_returns = recent_returns[:n]

            mean_reversion = -0.1 * recent_returns
            optimal_allocation += 0.2 * mean_reversion

        # Normalize to ensure sum is 1
        if np.sum(optimal_allocation) > 0:
            optimal_allocation = optimal_allocation / np.sum(optimal_allocation)
        else:
            optimal_allocation = np.ones(n) / n

        return optimal_allocation


# Helper functions for analysis and visualization

def calculate_factor_returns(env):
    """Calculate Fama-French style factor returns for momentum, value, and growth"""
    returns_history = np.array(env.history['returns'])
    signals_history = np.array(env.history['signals'])

    factor_returns = {
        'momentum': [],
        'value': [],
        'growth': []
    }

    for t in range(1, len(returns_history)):
        # Momentum factor (1M momentum signal)
        momentum_signal = signals_history[t-1, :, 0]
        top_momentum = np.argsort(momentum_signal)[-int(env.n_assets*0.1):]
        bottom_momentum = np.argsort(momentum_signal)[:int(env.n_assets*0.1)]
        momentum_return = np.mean(returns_history[t, top_momentum]) - np.mean(returns_history[t, bottom_momentum])

        # Value factor
        value_signal = signals_history[t-1, :, 2]
        top_value = np.argsort(value_signal)[-int(env.n_assets*0.1):]
        bottom_value = np.argsort(value_signal)[:int(env.n_assets*0.1)]
        value_return = np.mean(returns_history[t, top_value]) - np.mean(returns_history[t, bottom_value])

        # Growth factor
        growth_signal = signals_history[t-1, :, 3]
        top_growth = np.argsort(growth_signal)[-int(env.n_assets*0.1):]
        bottom_growth = np.argsort(growth_signal)[:int(env.n_assets*0.1)]
        growth_return = np.mean(returns_history[t, top_growth]) - np.mean(returns_history[t, bottom_growth])

        factor_returns['momentum'].append(momentum_return)
        factor_returns['value'].append(value_return)
        factor_returns['growth'].append(growth_return)

    return factor_returns


def calculate_asset_betas(env, factor_returns, window=12):
    """Calculate rolling betas of assets to factors"""
    returns_history = np.array(env.history['returns'])

    # Handle the case where not enough history is available
    if len(returns_history) <= window:
        return []

    factor_returns_array = np.column_stack([
        factor_returns['momentum'],
        factor_returns['value'],
        factor_returns['growth']
    ])



    betas = []

    for t in range(window, len(returns_history)):
        # Get window of returns
        asset_window = returns_history[t-window:t]
        factor_window = factor_returns_array[t-window:t]

        asset_betas = np.zeros((env.n_assets, 3))

        for n in range(env.n_assets):
            # Calculate beta for each asset to each factor
            for f in range(3):
                try:
                    cov = np.cov(asset_window[:, n], factor_window[:, f])[0, 1]
                    var = np.var(factor_window[:, f])
                    if var > 0:
                        asset_betas[n, f] = cov / var
                except (ValueError, IndexError):
                    # Handle potential errors due to insufficient data
                    asset_betas[n, f] = 0

        betas.append(asset_betas)

    return betas


def analyze_manager_performance(env):
    """Analyze performance of different manager types"""
    active_managers = env.agents['active']
    passive_managers = env.agents['passive']

    # Calculate performance metrics
    active_wealth = [agent.wealth_history for agent in active_managers]
    passive_wealth = [agent.wealth_history for agent in passive_managers]

    # Group active managers by type
    active_by_type = {
        'value': [],
        'growth': [],
        'blend': [],
        'esg': []
    }

    for i, agent in enumerate(active_managers):
        active_by_type[agent.manager_type].append(active_wealth[i])

    # Calculate average returns by type
    avg_returns = {
        'active': np.mean([np.array(w[-1])/w[0] - 1 for w in active_wealth if len(w) > 0]) if active_wealth else 0,
        'passive': np.mean([np.array(w[-1])/w[0] - 1 for w in passive_wealth if len(w) > 0]) if passive_wealth else 0
    }

    # Calculate for each manager type, handling empty lists
    for manager_type in ['value', 'growth', 'blend', 'esg']:
        wealth_list = active_by_type[manager_type]
        if wealth_list and any(len(w) > 0 for w in wealth_list):
            avg_returns[manager_type] = np.mean([np.array(w[-1])/w[0] - 1 for w in wealth_list if len(w) > 0])
        else:
            avg_returns[manager_type] = 0

    # Calculate sharpe ratios
    sharpe_ratios = {}
    for manager_type in ['active', 'passive', 'value', 'growth', 'blend', 'esg']:
        if manager_type == 'active':
            wealth_series = active_wealth
        elif manager_type == 'passive':
            wealth_series = passive_wealth
        else:
            wealth_series = active_by_type[manager_type]

        # Calculate returns series for each manager
        returns_series = []
        for w in wealth_series:
            if len(w) > 1:
                rets = [w[i]/w[i-1] - 1 for i in range(1, len(w))]
                returns_series.append(rets)

        # Calculate average Sharpe ratio
        if returns_series:
            all_returns = np.concatenate(returns_series)
            mean_return = np.mean(all_returns)
            std_return = np.std(all_returns)
            if std_return > 0:
                sharpe_ratios[manager_type] = mean_return / std_return * np.sqrt(12)  # Annualized
            else:
                sharpe_ratios[manager_type] = 0
        else:
            sharpe_ratios[manager_type] = 0

    # Calculate asset flow metrics
    retail_flows = np.array([agent.fund_investments for agent in env.agents['retail']])
    total_flows = np.sum(retail_flows, axis=0)

    # Split between active and passive
    active_flows = total_flows[:env.n_active]
    passive_flows = total_flows[env.n_active:]

    results = {
        'avg_returns': avg_returns,
        'sharpe_ratios': sharpe_ratios,
        'active_flows': active_flows,
        'passive_flows': passive_flows
    }

    return results


def plot_simulation_results(results):
    """Plot key results from the simulation"""
    # Extract data
    history = results['history']
    factor_returns = results['factor_returns']
    manager_analysis = results['manager_analysis']
    env = results['environment']

    # Create figure
    fig = plt.figure(figsize=(15, 20))

    # Plot 1: Asset prices
    ax1 = fig.add_subplot(4, 2, 1)
    prices = np.array(history['prices'])
    ax1.plot(prices)
    ax1.set_title('Asset Prices')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Price')

    # Plot 2: Factor returns
    ax2 = fig.add_subplot(4, 2, 2)
    ax2.plot(factor_returns['momentum'], label='Momentum')
    ax2.plot(factor_returns['value'], label='Value')
    ax2.plot(factor_returns['growth'], label='Growth')
    ax2.set_title('Factor Returns')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Return')
    ax2.legend()

    # Plot 3: Active vs Passive performance
    ax3 = fig.add_subplot(4, 2, 3)
    active_wealth = [agent.wealth_history for agent in env.agents['active']]
    passive_wealth = [agent.wealth_history for agent in env.agents['passive']]

    if active_wealth:
        avg_active = np.mean([np.array(w)/w[0] if len(w) > 0 else np.ones(env.max_steps+1) for w in active_wealth], axis=0)
        ax3.plot(avg_active, label='Active Managers')

    if passive_wealth:
        avg_passive = np.mean([np.array(w)/w[0] if len(w) > 0 else np.ones(env.max_steps+1) for w in passive_wealth], axis=0)
        ax3.plot(avg_passive, label='Passive Managers')

    ax3.set_title('Manager Performance (Wealth Growth)')
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Wealth (Normalized)')
    ax3.legend()

    # Plot 4: Manager types performance
    ax4 = fig.add_subplot(4, 2, 4)
    for manager_type in ['value', 'growth', 'blend', 'esg']:
        ax4.bar(manager_type, manager_analysis['avg_returns'][manager_type])
    ax4.set_title('Returns by Manager Type')
    ax4.set_ylabel('Return')

    # Plot 5: Sharpe ratios
    ax5 = fig.add_subplot(4, 2, 5)
    for manager_type in ['active', 'passive', 'value', 'growth', 'blend', 'esg']:
        ax5.bar(manager_type, manager_analysis['sharpe_ratios'][manager_type])
    ax5.set_title('Sharpe Ratios by Manager Type')
    ax5.set_ylabel('Sharpe Ratio')



    # Plot 6: Asset allocation over time
    ax6 = fig.add_subplot(4, 2, 6)
    if 'asset_allocations' in history and history['asset_allocations']:
        active_allocations = [alloc['active'] for alloc in history['asset_allocations']]
        if active_allocations:
            active_alloc_array = np.array(active_allocations)
            # Use only first 5 assets for clarity in the plot
            max_assets_to_show = min(5, active_alloc_array.shape[1])
            ax6.stackplot(range(len(active_allocations)),
                        [active_alloc_array[:, i] for i in range(max_assets_to_show)],
                        labels=[f'Asset {i}' for i in range(max_assets_to_show)])
    ax6.set_title('Active Managers Allocation (First 5 Assets)')
    ax6.set_xlabel('Time')
    ax6.set_ylabel('Allocation')
    ax6.legend(loc='upper left', fontsize='small')

    # Plot 7: Stock classification over time
    ax7 = fig.add_subplot(4, 2, 7)
    classification_history = []
    for t in range(len(history['signals'])):
        z = history['signals'][t]
        value_percentiles = np.argsort(np.argsort(z[:, 2])) / env.n_assets
        growth_percentiles = np.argsort(np.argsort(z[:, 3])) / env.n_assets
        vg_diff = value_percentiles - growth_percentiles

        value_count = np.sum(vg_diff > 0.3)
        growth_count = np.sum(vg_diff < -0.3)
        core_count = np.sum((vg_diff <= 0.3) & (vg_diff >= -0.3))

        classification_history.append([value_count, core_count, growth_count])

    classification_array = np.array(classification_history)
    ax7.stackplot(range(len(classification_history)),
                classification_array[:, 0],
                classification_array[:, 1],
                classification_array[:, 2],
                labels=['Value', 'Core', 'Growth'])
    ax7.set_title('Stock Classification Over Time')
    ax7.set_xlabel('Time')
    ax7.set_ylabel('Number of Stocks')
    ax7.legend()

    # Plot 8: Retail flows
    ax8 = fig.add_subplot(4, 2, 8)
    active_flows = np.sum(manager_analysis['active_flows'])
    passive_flows = np.sum(manager_analysis['passive_flows'])

    # Plot cumulative flows
    ax8.bar(['Active', 'Passive'], [active_flows, passive_flows])
    ax8.set_title('Retail Investor Flows')
    ax8.set_ylabel('Cumulative Flow')

    plt.tight_layout()

    return fig


# Function to run market simulation with interactive parameters
def run_market_simulation(config=None):
    """Run the market simulation with given configuration"""
    if config is None:
        config = {
            'n_assets': 20,
            'n_retail': 100,
            'n_active': 10,
            'n_passive': 5,
            'n_signals': 5,
            'risk_free_rate': 0.02,
            'dt': 1/12,
            'max_steps': 60,
            'active_fee': 0.015,
            'passive_fee': 0.005,
            'info_cost': 0.003,
            'retail_rationality': 5.0
        }

    # Initialize environment
    env = MarketEnvironment(**config)

    # Run simulation
    history = env.run_simulation()

    # Calculate factor returns
    factor_returns = calculate_factor_returns(env)

    # Calculate asset betas
    betas = calculate_asset_betas(env, factor_returns)

    # Analyze manager performance
    manager_analysis = analyze_manager_performance(env)

    # Return results
    results = {
        'history': history,
        'factor_returns': factor_returns,
        'asset_betas': betas,
        'manager_analysis': manager_analysis,
        'environment': env
    }

    return results


def print_summary_statistics(results):
    """Print key summary statistics from the simulation results"""
    # Extract data
    factor_returns = results['factor_returns']
    manager_analysis = results['manager_analysis']
    env = results['environment']

    print("\nSummary Statistics:")
    print("==================")

    print("\nAverage Returns:")
    for manager_type, ret in manager_analysis['avg_returns'].items():
        print(f"{manager_type}: {ret:.4f}")

    print("\nSharpe Ratios:")
    for manager_type, sharpe in manager_analysis['sharpe_ratios'].items():
        print(f"{manager_type}: {sharpe:.4f}")

    # Factor correlations
    if factor_returns['momentum'] and factor_returns['value'] and factor_returns['growth']:
        factor_array = np.column_stack([
            factor_returns['momentum'],
            factor_returns['value'],
            factor_returns['growth']
        ])

        factor_corr = np.corrcoef(factor_array.T)
        print("\nFactor Correlations:")
        print("Momentum-Value:", factor_corr[0, 1])
        print("Momentum-Growth:", factor_corr[0, 2])
        print("Value-Growth:", factor_corr[1, 2])

    # Asset reclassification frequency
    classification_changes = 0
    prev_labels = None

    for t in range(len(results['history']['signals'])):
        z = results['history']['signals'][t]
        value_percentiles = np.argsort(np.argsort(z[:, 2])) / env.n_assets
        growth_percentiles = np.argsort(np.argsort(z[:, 3])) / env.n_assets
        vg_diff = value_percentiles - growth_percentiles

        current_labels = np.zeros(env.n_assets, dtype=object)
        current_labels[vg_diff > 0.3] = 'value'
        current_labels[vg_diff < -0.3] = 'growth'
        current_labels[(vg_diff <= 0.3) & (vg_diff >= -0.3)] = 'core'

        if prev_labels is not None:
            classification_changes += np.sum(prev_labels != current_labels)

        prev_labels = current_labels

    avg_changes_per_period = classification_changes / (len(results['history']['signals']) - 1) if len(results['history']['signals']) > 1 else 0
    print(f"\nAverage asset reclassifications per period: {avg_changes_per_period:.2f}")


# Interactive simulation with sliders for parameters
def interactive_simulation():
    """Create interactive sliders for simulation parameters"""
    # Define widgets for parameters
    n_assets_slider = widgets.IntSlider(
        value=20, min=5, max=50, step=5,
        description='Assets:',
        continuous_update=False
    )

    n_retail_slider = widgets.IntSlider(
        value=100, min=10, max=500, step=10,
        description='Retail Investors:',
        continuous_update=False
    )

    n_active_slider = widgets.IntSlider(
        value=8, min=1, max=20, step=1,
        description='Active Managers:',
        continuous_update=False
    )


    n_passive_slider = widgets.IntSlider(
        value=4, min=1, max=20, step=1,
        description='Passive Managers:',
        continuous_update=False
    )

    rf_slider = widgets.FloatSlider(
        value=0.02, min=0.0, max=0.1, step=0.005,
        description='Risk-Free Rate:',
        continuous_update=False
    )

    active_fee_slider = widgets.FloatSlider(
        value=0.015, min=0.001, max=0.05, step=0.001,
        description='Active Fee:',
        continuous_update=False
    )

    passive_fee_slider = widgets.FloatSlider(
        value=0.005, min=0.0001, max=0.02, step=0.0005,
        description='Passive Fee:',
        continuous_update=False
    )

    info_cost_slider = widgets.FloatSlider(
        value=0.003, min=0.0001, max=0.02, step=0.0005,
        description='Info Cost:',
        continuous_update=False
    )

    rationality_slider = widgets.FloatSlider(
        value=5.0, min=0.5, max=20.0, step=0.5,
        description='Retail Rationality:',
        continuous_update=False
    )

    time_steps_slider = widgets.IntSlider(
        value=60, min=12, max=120, step=12,
        description='Time Steps:',
        continuous_update=False
    )

    run_button = widgets.Button(
        description='Run Simulation',
        button_style='success',
        tooltip='Click to run the simulation with selected parameters',
        icon='play'
    )

    output = widgets.Output()

    # Layout for the sliders and output
    parameter_box = widgets.VBox([
        widgets.HBox([n_assets_slider, n_retail_slider]),
        widgets.HBox([n_active_slider, n_passive_slider]),
        widgets.HBox([rf_slider, time_steps_slider]),
        widgets.HBox([active_fee_slider, passive_fee_slider]),
        widgets.HBox([info_cost_slider, rationality_slider]),
        run_button
    ])

    # Function to execute when run button is clicked
    def on_run_button_click(b):
        with output:
            output.clear_output()
            print("Starting simulation...")

            config = {
                'n_assets': n_assets_slider.value,
                'n_retail': n_retail_slider.value,
                'n_active': n_active_slider.value,
                'n_passive': n_passive_slider.value,
                'n_signals': 5,  # Fixed parameter
                'risk_free_rate': rf_slider.value,
                'dt': 1/12,  # Fixed parameter (monthly time steps)
                'max_steps': time_steps_slider.value,
                'active_fee': active_fee_slider.value,
                'passive_fee': passive_fee_slider.value,
                'info_cost': info_cost_slider.value,
                'retail_rationality': rationality_slider.value
            }

            results = run_market_simulation(config)
            fig = plot_simulation_results(results)
            plt.show()
            print_summary_statistics(results)

    run_button.on_click(on_run_button_click)

    # Create the UI layout
    ui = widgets.VBox([parameter_box, output])

    return ui


# Main function to run and visualize the simulation
def main():
    print("Interactive Multi-Agent Reinforcement Learning (MARL) Model")
    print("with Active and Passive Investors in a Multi-Asset Market\n")
    print("Adjust parameters using the sliders and click 'Run Simulation' to start.")

    ui = interactive_simulation()
    display(ui)

    return ui


if __name__ == "__main__":
    main()


Interactive Multi-Agent Reinforcement Learning (MARL) Model
with Active and Passive Investors in a Multi-Asset Market

Adjust parameters using the sliders and click 'Run Simulation' to start.
